# FIT5196 Task 1 in Assessment 1
#### Student Name: Vyom Chauhan
#### Student ID: 30830192

Date: 13/09/2020

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

</font>

This task is to extract the english tweets from the given twitter data and output them as a file in xml format. Firstly we read all the files from the directory 30830192 and append it to a string. Next we use the findall function from the re library to extract text, date and id of the tweets. Regular expression r'"text":".*?"' - extracts all strings beginning with "text":" and having whatever characters and ending with ". Regular expression r'"id":"\d{19}"' - extracts all strings beginning with "id":" and having 19 digits and ending with ". Regular expression r'"created_at":"\d{4}-\d{2}-\d{2}' - extracts all strings beginning with "created_at":" and having date in the format xxxx-xx-xx. Next we clean the text, date and id and mofify it to the desired xml format and classify them extracting only the english texts. Next we create a dictionary by zipping the three lists and then update the dictionary using a for loop. We store the final output as a string  modified into the xml format and then write it to the output file. On parsing the output file it matches the desired xml format as required for the task.

In [ ]:
# importing the required libraries
import os,re
from langid import classify

In [ ]:
# variable to store the text from files
text = ""
# accessing the directory '30830192' which has tweet data
for root, dirs, files in os.walk("30830192"):
    # accessing the files from directory part1
    for file in files:
        with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
            # reading the file
            data = f.read()
            # appending file data to a string
            text += data

In [ ]:
# extracts all strings beginning with "text":" and having whatever characters and ending with "
tweet = re.findall(r'"text":".*?"', text)
# extracts all strings beginning with "id":" and having 19 digits and ending with "
data_id = re.findall(r'"id":"\d{19}"', text)
# extracts all strings beginning with "created_at":" and having date in the format xxxx-xx-xx
date = re.findall(r'"created_at":"\d{4}-\d{2}-\d{2}', text)

In [ ]:
# lists to store english tweets and their corresponding dates and ids
tweet_en = []
date_en = []
id_en = []
# cleaning the data and extracting english tweets
for i in range(len(tweet)):
    # removing "text":" and extra "
    tweet[i] = tweet[i][8:-1]
    # removing "created_at":" 
    date[i] = date[i][14:]
    # removing "id" and extra "
    data_id[i] = data_id[i][5:]
    # adding special characters for xml format
    mod = "\"" + date[i] + "\""
    date[i] = "tweets date=" + mod
    data_id[i] = "tweet id=" + data_id[i]
    # extracting english text and their corresponding date and id while removing duplicate id's
    if classify(tweet[i])[0] == 'en' and data_id[i] not in id_en:
        tweet_en.append(tweet[i])
        date_en.append(date[i])
        id_en.append(data_id[i])

In [ ]:
# creating a nested dictionary with date as the parent key and id as the child key with the tweet as value
data_dic = {str(a): {str(b): str(c)} for a, b, c in zip(date_en, id_en, tweet_en)}

In [ ]:
# editing data_dic since it only contains one tweet for a date  
for key, nested_dict in data_dic.items():
    for (d, i, t) in zip(date_en, id_en, tweet_en):
        if key == d:
            # updating data_dic if the tweet id doesn't exist for that date
            if not(i in nested_dict):
                data_dic[key].update({i:t})         

In [ ]:
# converting the dictionary into the xml format
final_xml = "<?xml version=\"1.0\" encoding=\"UTF-8\"?>" + "\n" + "<data>" + "\n"
for date, nested_dict in data_dic.items():
    final_xml += "<" + date + ">\n"
    for tweet_id in sorted(nested_dict):
        final_xml += "<" + tweet_id + ">" + nested_dict[tweet_id] + "</tweet>\n"
    final_xml += "</tweets>\n"
final_xml += "</data>"

In [ ]:
# creating xml file for writing
file = open("30830192.xml", "w", encoding='utf-8') 
# writing to xml file
file.write(final_xml) 
# closing file object
file.close()

In [ ]:
# with open('30830192.xml', encoding='utf-8') as fd:
#     doc = xmltodict.parse(fd.read())
# doc